In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
pd.options.display.max_colwidth = 1000

In [2]:
# Permet d'obtenir la liste d'url de la page i
def get_url_list(i):
    url = "https://www.asn.fr/l-asn-controle/actualites-du-controle/installations-nucleaires/arret-de-reacteurs-de-centrales-nucleaires?page="+str(i)
    response = requests.get(url)
    if not response.status_code == 200:
        print(f"La requête n°{i} a échoué avec le code de statut :", response.status_code)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Sélectionnez l'élément <a> avec la classe "Teaser-titleLink"
    elements_a = soup.find_all('a', class_='Teaser-titleLink')

    link_list = []
    for element in elements_a:
        link_list.append('https://www.asn.fr' + element['href'])
        
    return link_list


# Trouve une liste des mots commençant par une majuscule
def maj(phrase):
    mots = re.findall(r'\b[A-Z][A-Za-z-]*\b', phrase)
    return mots

# Retourne le premier chiffre de la phrase
def num(phrase):
    match = re.search(r'\d', phrase)
    
    if match:
        # Renvoyer le premier chiffre trouvé
        return int(match.group())
    else:
        # Renvoyer None si aucun chiffre n'est trouvé
        return None

# Trouve les informations concernant le réacteur d'après l'url donnée
def get_infos(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trouver l'élément <div> avec la classe "AsideBlock-reactor"
        reactor_div = soup.find('div', class_='AsideBlock-reactor')
        
        begin_date, end_date = '', ''
        
        if reactor_div:
            # Trouver l'élément <p> avec la classe "AsideBlock-reactor-shutdown-begin-date"
            begin_date_element = reactor_div.find('p', class_='AsideBlock-reactor-shutdown-begin-date')
            begin_date = begin_date_element.find('b').text.strip()

            # Trouver l'élément <p> avec la classe "AsideBlock-reactor-shutdown-end-date"
            end_date_element = reactor_div.find('p', class_='AsideBlock-reactor-shutdown-end-date')
            end_date = end_date_element.find('b').text.strip()
        
        p1, p2 = "", ""
        
        paragraph_element = soup.find('div', class_='ezrichtext-field').find('p')
        if paragraph_element :
            p1 = paragraph_element.text.strip()

        paragraph_element = soup.find('div', class_='RichText').find('p')
        if paragraph_element:
            p2 = paragraph_element.text.strip()
        
        text = p1 + p2

        # Afficher les dates de début et de fin
    else:
        print("La requête a échoué avec le code de statut :", response.status_code)
        
    reactor_number = num(text)
    
    if maj(text) and len(maj(text)) > 1:
        reactor_name = maj(text)[1]
    else:
        print(maj(text))
        reactor_name = 'None'
        
    return [reactor_number, reactor_name, begin_date, end_date, url]


In [ ]:
info_list = []
total_list = []

for i in range(60):
    total_list += get_url_list(i)

for url in total_list:
    try:
        info_list.append(get_infos(url))
    except IndexError:
        None
        
df = pd.DataFrame(info_list, columns=['Number','Name', 'Start', 'End', 'Url'])